In [1]:
%reload_ext autoreload
%autoreload 2
from IPython.display import Math
from tqdm import tqdm
import sys
import os
parent_path = os.path.abspath("..")
if parent_path not in sys.path:
    sys.path.append(parent_path)

# For a quick config:
# import logging
# logging.basicConfig(
#     format="%(asctime)s [%(levelname)s] [%(name)s]: %(message)s",
#     level=logging.DEBUG
# )

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import sympy
import numpy as np
import numpy.typing as npt
import typing
from typing import Callable, Dict, List, Optional, Tuple, Union, Iterable
import qutip # type: ignore

from src.quantum_utils import measure_povm, sic_povm, mub_povm, random_rank1_povm
from src.qelm_utils import analyze_biasvar_vs_nstates, analyze_biasvar_vs_statistics

from src.shadow_tomography import frame_operator
from src.utils import pp_matrix
from src import QELM

def make_cube_states():
    # compute the vertices of a unit cube
    vertices = np.array([[x, y, z] for x in [-1, 1] for y in [-1, 1] for z in [-1, 1]], dtype=float)
    # normalize the vertices to lie on the unit sphere
    vertices /= np.linalg.norm(vertices, axis=1)[:, np.newaxis]
    # for each vector in vertices, compute the density matrix (I + v\cdot\sigma)/2
    sigma_matrices = [qutip.sigmax(), qutip.sigmay(), qutip.sigmaz()]
    cube_states = []
    for v in vertices:
        sigma_v = sum(v[i] * sigma_matrices[i] for i in range(3))
        dm = (qutip.qeye(2) + sigma_v) / 2
        cube_states.append(dm)
    return cube_states
cube_states = make_cube_states()
sic_states = [2 * effect for effect in sic_povm()]
mub_states = [3 * effect for effect in mub_povm()]

In [ ]:
%%timeit
random_states_qutip = [qutip.rand_ket(2) for _ in range(10000)]

1.59 s ± 9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def gaussian_random_single_qubit_states(num_states):
    """
    Generate 'num_states' Haar-random single-qubit pure states
    via normalized complex Gaussian vectors.
    Returns a (num_states, 2) array of complex amplitudes.
    """
    # Draw random complex vectors: real and imaginary parts ~ N(0,1)
    psi = np.random.randn(num_states, 2) + 1j * np.random.randn(num_states, 2)
    # Normalize each row
    norms = np.linalg.norm(psi, axis=1, keepdims=True)
    psi /= norms
    return psi

In [5]:
%%timeit
random_states_qutip = gaussian_random_single_qubit_states(10000)

803 μs ± 3.62 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
statistics = 100
random_states = [qutip.rand_ket(2) for _ in range(100)]
povm = sic_povm()

measure_povm(states=random_states, povm=povm, statistics=statistics, return_frequencies=True).sum(axis=1)

array([23.86, 24.71, 24.28, 27.15])

In [25]:
statistics = 100
random_states = [qutip.rand_ket(2) for _ in range(100)]
povm = random_rank1_povm(dim=2, num_outcomes=10)

print(measure_povm(states=random_states, povm=povm, statistics=statistics, return_frequencies=True).sum(axis=1) / statistics)
[round(np.trace(effect.full()).real, 3) / 2 for effect in povm]

[0.0596 0.1406 0.1436 0.2341 0.049  0.0785 0.0359 0.0501 0.0455 0.1631]


[0.0585, 0.137, 0.1285, 0.2415, 0.055, 0.086, 0.036, 0.0525, 0.044, 0.1605]

$1+\frac12$